### 0. Imports

In [16]:
# Data transformation
# ---------------------------------------
import pandas as pd
import polars as pl
import numpy as np

# API calls
# ---------------------------------------
import requests

# Scraping
# ---------------------------------------
from bs4 import BeautifulSoup


# Aynchronicity
# ---------------------------------------
import asyncio
import aiohttp


import json


import time


import re

from tqdm import tqdm

from joblib import Parallel, delayed

import sys

if sys.platform == 'win32':
    loop = asyncio.ProactorEventLoop()
    asyncio.set_event_loop(loop)

sys.path.append("..")

import src.soporte_eda as se

# 1. Introducción - Limpieza datos Movies Database

Este notebook detalla la limpieza de los datos previa a la subida en base de datos.

# 2. Importacion y exploracion

## 2.1 Detalles artistas

In [5]:
detalles_artistas = pd.read_csv("../data/raw/detalles_artistas.csv")
detalles_artistas.sample(5)

,actr,año_nac,conocido_por,que_hace,premios
717,Anthony Herrera,1944,"'As the World Turns', 'American Playhouse', 'S...","Actor, Director, Writer",3 nominations
6656,Mitsuki Higuchi,9999,'Kôshoku kunoichi ninpô-chô: Vâjin sunaipâ - b...,Actor,no tiene premios
8535,Sheryl Lee,1967,"'Twin Peaks: Fire Walk with Me', 'Vampires', '...","Actor, Director, Script and Continuity Department",no tiene premios
8454,Shan Kelly,9999,"'Arma letal', 'Infieles', 'Akeelah contra todo...",Actor,no tiene premios
393,Alyss Winkler,9999,"'Space Babes from Outer Space', 'Hellfire', 'T...","Costume Designer, Costume Department, Actress",no tiene premios


Exploración del dataset

In [15]:
se.exploracion_dataframe(detalles_artistas)

El número de datos es 9907 y el de columnas es 5

 ..................... 

5 filas aleatorias del dataframe son:


,actr,año_nac,conocido_por,que_hace,premios
4423,Jillian Nusbaum,9999,'Eating Out',Actor,no tiene premios
1479,Carla Martinez,1956,"'Darna', 'Wildflower', 'Artikulo 247', 'Kristine'",Actress,no tiene premios
7918,Robert Shannon,9999,"'Candy', 'The Information', 'SO19', 'Forbidden...","Actor, Producer, Director",no tiene premios
2157,Dan Harrison,9999,'Older Than America',"2008, Not Rated, 1h 50m",4 wins & 5 nominations
3885,Isabella Lafin,9999,"'O Jantar', 'O Tesouro Perdido', 'No Limite', ...",Actor,no tiene premios



 ..................... 

Los tipos de las columnas y sus valores únicos son:


,tipo_dato,conteo
actr,object,9907
año_nac,int64,115
conocido_por,object,9301
que_hace,object,1076
premios,object,872



 ..................... 

Los duplicados que tenemos en el conjunto de datos son: 0

 ..................... 

Los nulos que tenemos en el conjunto de datos son:


...

,count,pct
conocido_por,,
'No hay datos sobre porqué es conocido',170,1.7
'Panama',9,0.1
'Specimen 0625c',8,0.1
'2013: Holocaust',7,0.1
'A Boy Named Bellamy',7,0.1


La columna QUE_HACE tiene 1076 valores únicos, de los cuales los primeros son:


,count,pct
que_hace,,
Actor,2355,23.8
Actress,1754,17.7
"Actor, Director, Writer",269,2.7
"Actor, Producer, Director",230,2.3
"Actress, Soundtrack",202,2.0


La columna PREMIOS tiene 872 valores únicos, de los cuales los primeros son:


,count,pct
premios,,
no tiene premios,6075,61.3
1 nomination,483,4.9
1 win,409,4.1
2 nominations,185,1.9
1 win & 1 nomination,171,1.7


Se observa que:
- Hay valores erróneos en qué hace, diferentes a las profesiones. Son dificiles de limpiar, así que se dejan así de momento.


- Al menos el 50% de los Actores tienen una fecha de nacimiento igual a 9999, que equivaldría a nulo.
- es_conocido_por tiene nulos marcados en texto; 'No hay datos sobre porqué es conocido'
- de forma similar, premios tiene los 0 marcados como 'no tiene premios'

Corrigiendo estos fallos:

In [27]:
repl_dict = {"conocido_por":{'No hay datos sobre porqué es conocido':np.nan},
            "premios":{'no tiene premios':0},
            "año_nac":{9999:np.nan}
             }
detalles_artistas.replace({"conocido_por":{'No hay datos sobre porqué es conocido':np.nan}}, inplace=True)
detalles_artistas.replace({"premios":{'no tiene premios':0}}, inplace=True)
detalles_artistas.replace({"año_nac":{9999:np.nan}}, inplace=True)
detalles_artistas

,actr,año_nac,conocido_por,que_hace,premios
0,A. Gunaseelan,NaN,'Mandela',"Casting Department, Actress, Casting Director",1 nomination
1,A. Michelle Harleston,NaN,"'Murder by Numbers', 'The Mark of a Killer', '...",Actor,0
2,A.C. Peterson,NaN,"'Narc', 'Shanghai Noon', 'Shooter', 'The Last ...",Actor,0
3,A.V.S. Subramanyam,1957.0,"'Roommates', 'Uncle', 'Ori Nee Prema Bangaramk...","Actor, Director, Writer",0
4,Øyvind Øvrebø,NaN,"'Tingen, Edderkoppen', 'Demenskoret', 'Kristia...","Actress, Additional Crew",0
...,...,...,...,...,...
9902,Zsolt Unoka,NaN,"'Panic', 'Rám csaj még nem volt ilyen hatással...",Actor,0
9903,Zsuzsanna Ujj,1959.0,"'Rám csaj még nem volt ilyen hatással', 'Sade ...","Actress, Second Unit Director or Assistant Dir...",0
9904,Zu Buklaha,NaN,'Kobieta Plus Drzewo',"Actor, Producer",0
9905,Zuza Tehanu,NaN,"'The Viking Revenge', 'Christmas Party', 'Cybe...",Actor,0


Tal vez sea necesario adicionalmente explotar las peliculas por las que cada artista es conocido.

## 2.2 Detalles peliculas

In [7]:
detalles_peliculas = pd.read_csv("../data/raw/detalles_peliculas.csv")
detalles_peliculas.sample(5)

,titulo,cali_imdb,direc_imbd,guion_imbd,argum_imbd,durac_imbd,id_pelicula
1316,Sessions of the Mind,6.4,Uisdean Murray,Uisdean Murray,"A tale of two women, Jemima, who attempted to ...",47min,tt1339166
1361,After the Third Bell,6.6,Ajay Govind,Ajay Govind,The death of an actor in front of a live audie...,1h 32min,tt3526430
488,Skin Deep,NaN,Desconocido,Desconocido,Desconocido,9m,tt2008636
588,Just Like You...,5.8,Thomas Norman,Desconocido,Desconocido,3m,tt2318569
123,"English, August",7.6,Dev Benegal,Dev BenegalUpamanyu Chatterjee,"Agastya Sen (Rahul Bose), nicknamed ""English, ...",1h 58min,tt0109732


In [28]:
se.exploracion_dataframe(detalles_peliculas)

El número de datos es 1453 y el de columnas es 7

 ..................... 

5 filas aleatorias del dataframe son:


,titulo,cali_imdb,direc_imbd,guion_imbd,argum_imbd,durac_imbd,id_pelicula
1116,A Boy Named Bellamy,NaN,Jess Farr,Israel AriasJess Farr,After witnessing his mother commit a serious c...,10min,tt3888418
359,The Santa Clause 2,5.7,Desconocido,Desconocido,Desconocido,1h 44min,tt0304669
494,Chiller's Dare 2 Direct Short Film Festival 2007,NaN,Brian Malone,Brian Malone,Winners of Chiller's Dare 2 Direct Short Film ...,30m,tt2043811
898,Istanbul Kanatlarimin Altinda,7.0,Mustafa Altioklar,Mustafa Altioklar,Desconocido,1h 59min,tt0272142
512,Dead Ahead,4.6,Desconocido,Desconocido,Desconocido,Desconocido,tt2122322



 ..................... 

Los tipos de las columnas y sus valores únicos son:


,tipo_dato,conteo
titulo,object,1444
cali_imdb,float64,81
direc_imbd,object,858
guion_imbd,object,906
argum_imbd,object,682
durac_imbd,object,231
id_pelicula,object,1453



 ..................... 

Los duplicados que tenemos en el conjunto de datos son: 0

 ..................... 

Los nulos que tenemos en el conjunto de datos son:


...

,count,pct
argum_imbd,,
Desconocido,772,53.1
This psychological action drama was made in tribute to the Czech school for stuntmen: thus the plot line is subordinated to showcasing their artistic sport. The psychological level depicts a family tragedy in which a hero's world caves in after the loss of his son. Thanks to a group of stuntmen he is able to begin again.,1,0.1
Two students produce a documentary about the demolition of their high school.,1,0.1
"Baasha, a gangster, controls a whole village with an iron fist. Pazhanisamy, who relocates to the village, questions the riots and earns Baasha's enmity. Can he defeat Baasha and save the people?",1,0.1
"As punishment for participating in a contest TV show, a nun is sent to a small town. The boss and aspirant to be the mayor, who refuses to have children vaccinated and go to school, opposes the presence of the new arrival of the nun, but she conquers the people and help to rescue the kidnapped daughter of the boss.",1,0.1


La columna DURAC_IMBD tiene 231 valores únicos, de los cuales los primeros son:


,count,pct
durac_imbd,,
Desconocido,231,15.9
1h 30min,58,4.0
1h 40min,28,1.9
1h 35min,26,1.8
15min,25,1.7


La columna ID_PELICULA tiene 1453 valores únicos, de los cuales los primeros son:


,count,pct
id_pelicula,,
tt27215062,1,0.1
tt0320656,1,0.1
tt0318803,1,0.1
tt0320503,1,0.1
tt0330019,1,0.1


Parece que aqui la unicas correcciones necesarias serían:
- La conversion de duracion a minutos. Se deja para más tarde, la subida a BDD es más prioritaria.
- El cambio de los valores 'desconocido' a nulo.

In [31]:
detalles_peliculas.replace({"Desconocido":np.nan}, inplace=True)

## 2.3 Pelicula artista

In [6]:
pelicula_artista = pd.read_csv("../data/raw/pelicula_artista.csv")
pelicula_artista.sample(5)

,Movie ID,Actriz_Actor
2958,tt0273300,Alan Gryfe
2613,tt1121949,Greg Thompson
274,tt1999895,Gemma Giddings
10462,tt2314254,Sin datos
8149,tt1453949,Sin datos


In [32]:
se.exploracion_dataframe(pelicula_artista)

El número de datos es 14529 y el de columnas es 2

 ..................... 

5 filas aleatorias del dataframe son:


,Movie ID,Actriz_Actor
118,tt11125620,Oscar Isaac
342,tt0340227,Kane Kosugi
47,tt0230041,Verne Troyer
13067,tt18688348,Mykhailo Illienko
1740,tt2316550,Adzwa Aurelline



 ..................... 

Los tipos de las columnas y sus valores únicos son:


,tipo_dato,conteo
Movie ID,object,1452
Actriz_Actor,object,9909



 ..................... 

Los duplicados que tenemos en el conjunto de datos son: 3298

 ..................... 

Los nulos que tenemos en el conjunto de datos son:


...

,count,unique,top,freq
Movie ID,14529,1452,tt0929793,20
Actriz_Actor,14529,9909,Sin datos,4094



 ..................... 

Los valores que tenemos para las columnas categóricas son: 
La columna MOVIE ID tiene 1452 valores únicos, de los cuales los primeros son:


,count,pct
Movie ID,,
tt0929793,20,0.1
tt0113865,10,0.1
tt10774576,10,0.1
tt0110660,10,0.1
tt0110363,10,0.1


La columna ACTRIZ_ACTOR tiene 9909 valores únicos, de los cuales los primeros son:


,count,pct
Actriz_Actor,,
Sin datos,4094,28.2
Shakti Kapoor,8,0.1
Anupam Kher,8,0.1
Gulshan Grover,6,0.0
Mohnish Behl,6,0.0


Aquí la nueva codificacion de valor nulo es 'Sin datos'.

In [33]:
pelicula_artista.replace({"Sin datos":np.nan}, inplace=True)

## 2.4 Peliculas

In [8]:
peliculas = pd.read_csv("../data/raw/peliculas.csv")
peliculas.sample(5)

,Tipo,Titulo,Año,Mes,Id,generos
1139,Movie,Saving Dreams,2016,9,tt4813906,Thriller
773,Short,Locker,2019,5,tt10767566,Horror
55,Movie,Tiempo de valientes,2005,9,tt0462570,Action
289,Short,Terror in the Hot Tubs,1992,5,tt0105567,Comedy
998,TV Movie,Ömer Seyfettin: Perili kösk,2005,0,tt10345862,Thriller


In [34]:
se.exploracion_dataframe(peliculas)

El número de datos es 1453 y el de columnas es 6

 ..................... 

5 filas aleatorias del dataframe son:


,Tipo,Titulo,Año,Mes,Id,generos
854,Short,Erecting A Monster,2022,5,tt19367438,Horror
462,Short,Astral,2019,5,tt10100940,Fantasy
945,Short,Running Without Sound,2004,6,tt0410536,Romance
136,Movie,Habit,1997,11,tt0113241,Drama
238,Movie,"Meine Mutter, mein Bruder und ich!",2008,5,tt0935101,Drama



 ..................... 

Los tipos de las columnas y sus valores únicos son:


,tipo_dato,conteo
Tipo,object,4
Titulo,object,1444
Año,int64,35
Mes,int64,13
Id,object,1453
generos,object,8



 ..................... 

Los duplicados que tenemos en el conjunto de datos son: 0

 ..................... 

Los nulos que tenemos en el conjunto de datos son:


...

,count,pct
Mes,,
3,162,11.1
10,153,10.5
1,129,8.9
4,128,8.8
5,119,8.2


La columna ID tiene 1453 valores únicos, de los cuales los primeros son:


,count,pct
Id,,
tt27262238,1,0.1
tt0320656,1,0.1
tt0318803,1,0.1
tt0320503,1,0.1
tt0330019,1,0.1


La columna GENEROS tiene 8 valores únicos, de los cuales los primeros son:


,count,pct
generos,,
Horror,405,27.9
Thriller,324,22.3
Drama,177,12.2
Comedy,161,11.1
Mystery,160,11.0


La variable mes tiene 13 valores únicos, así que hay que comprobarlos todos.

In [36]:
peliculas["Mes"].value_counts()

Mes
3     162
10    153
1     129
4     128
5     119
9     116
6     111
2     101
0      95
11     92
8      89
12     79
7      79
Name: count, dtype: int64

Hay mes 0, así que se asume que es un valor nulo.

In [41]:
peliculas.loc[peliculas["Mes"]==0, "Mes"] = np.nan

También hay años 9999. Se asume que son nulos.

In [42]:
peliculas.loc[peliculas["Año"]==0, "Año"] = np.nan

Se observa que los títulos de las peliculas, que deberían ser únicos, en algunos casos más frecuentes aparecen 2 veces. Conviene realizar otra comprobacion:

In [50]:
peliculas_doble = (peliculas["Titulo"].value_counts()[peliculas["Titulo"].value_counts() > 1]).index.to_list()
peliculas[peliculas["Titulo"].isin(peliculas_doble)].sort_values("Titulo")

,Tipo,Titulo,Año,Mes,Id,generos
803,Short,Clowning Around,2020.0,4,tt12235836,Horror
607,Short,Clowning Around,2012.0,2,tt2318433,Horror
496,Short,Dead End,2007.0,9,tt2057385,Horror
577,Short,Dead End,2011.0,6,tt2005193,Horror
657,Movie,Follow,2015.0,9,tt4396584,Horror
1082,Short,Follow,2013.0,3,tt2697770,Thriller
636,Short,Guilt,2014.0,8,tt3614870,Horror
1249,Short,Guilt,2019.0,9,tt10499300,Thriller
609,Short,Hush,2012.0,2,tt2318535,Horror
1022,Short,Hush,2008.0,9,tt1315045,Thriller


Debe tratarse de Remakes o cosas similares, ya que difieren en id y en fecha, así que se dejan tal cual.

# 3. Subida datos limpios a `datos/cleaned`

In [52]:
detalles_artistas.to_csv("../data/cleaned/detalles_artistas.csv")
detalles_peliculas.to_csv("../data/cleaned/detalles_peliculas.csv")
pelicula_artista.to_csv("../data/cleaned/pelicula_artista.csv")
peliculas.to_csv("../data/cleaned/peliculas.csv")